In [1]:
import argparse
import re
import networkx as nx
import scipy as scipy
import matplotlib.pyplot as plt # for visualizing GMLs
import os
import itertools
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn.metrics as sklearn_metrics
from sklearn.metrics import rand_score,mutual_info_score,adjusted_rand_score,adjusted_mutual_info_score

In [ ]:
# Mapping Clusters of Orthologous Genes (COGs) between two pan-genome gene graphs

In [117]:
# define functions to read in graphs with metadata

def conv_list(maybe_list):
    if not isinstance(maybe_list, list):
        maybe_list = [maybe_list]
    return (maybe_list)

def update_sid(sid, member_count):
    sid = sid.split("_")
    sid[0] = str(member_count + int(sid[0].replace("'", "")))
    return ("_".join(sid))

def del_dups(iterable):
    seen = {}
    for f in iterable:
        seen[f] = None
    return (list(seen.keys()))

def load_graphs(graph_files, n_cpu=1):
    for graph_file in graph_files:
        if not os.path.isfile(graph_file):
            print("Missing:", graph_file)
            raise RuntimeError("Missing graph file!")

    graphs = [nx.read_gml(graph_file) for graph_file in graph_files]
    isolate_names = list(
        itertools.chain.from_iterable(
            [G.graph['isolateNames'] for G in graphs]))

    member_count = 0
    node_count = 0
    id_mapping = []
    for i, G in enumerate(graphs):
        id_mapping.append({})
        # relabel nodes to be consecutive integers from 1
        mapping = {}
        for n in G.nodes():
            mapping[n] = node_count
            node_count += 1
        G = nx.relabel_nodes(G, mapping, copy=True)

        # set up edge members and remove conflicts.
        for e in G.edges():
            G[e[0]][e[1]]['members'] = [
                m + member_count for m in conv_list(G[e[0]][e[1]]['members'])
            ]

        # set up node parameters and remove conflicts.
        max_mem = -1
        for n in G.nodes():
            ncentroids = []
            for sid in G.nodes[n]['centroid'].split(";"):
                nid = update_sid(sid, member_count)
                id_mapping[i][sid] = nid
                if "refound" not in nid:
                    ncentroids.append(nid)
            G.nodes[n]['centroid'] = ncentroids
            new_ids = set()
            for sid in conv_list(G.nodes[n]['seqIDs']):
                nid = update_sid(sid, member_count)
                id_mapping[i][sid] = nid
                new_ids.add(nid)
            G.nodes[n]['seqIDs'] = new_ids
            G.nodes[n]['protein'] = del_dups(G.nodes[n]['protein'].replace(
                '*', 'J').split(";"))
            G.nodes[n]['dna'] = del_dups(G.nodes[n]['dna'].split(";"))
            G.nodes[n]['lengths'] = conv_list(G.nodes[n]['lengths'])
            G.nodes[n]['longCentroidID'][1] = update_sid(
                G.nodes[n]['longCentroidID'][1], member_count)
            G.nodes[n]['members'] = [m + member_count for m in conv_list(G.nodes[n]['members'])]
            max_mem = max(max_mem, max(G.nodes[n]['members']))

        member_count = max_mem + 1
        graphs[i] = G

    return graphs, isolate_names, id_mapping

In [128]:
# then replace individual run seqID with seqID from run of all data
def indSID_to_allSID(G, gid_map):
    
    for node in G.nodes():
        node_GIDs = G.nodes[node].get("seqIDs", "")  
        if node_GIDs:
            updated_GIDs = [
                gid_map.get(gid.strip(), "error")  # put 'error' if not found
                for gid in node_GIDs
            ]
            G.nodes[node]["seqIDs"] = updated_GIDs

    return G



def get_seqIDs_in_nodes(G):  

    # get individual gene IDs as keys, and the cluster each geneID belongs to as its value
    dictionary = {}
    total_geneids = 0
    total_nodes = 0
    
    for node in G.nodes():
        gene_ids = G.nodes[node].get("seqIDs")
        #print(f"graph: 1, node_index_id: {node}, node_group_id: {node_group}")
        
        #print(f"gene ids: {gene_ids}")
        #print(f"len gene ids: {len(gene_ids)}")
        #print(f"node: {node}")
        
        total_geneids += len(gene_ids)
        total_nodes += 1
        
        
        for GID in gene_ids:
            dictionary[GID] = node
     
    print(f"total seqids in all nodes: {total_geneids}")
    print(f"total nodes in graph: {total_nodes}")
        
    return dictionary

In [129]:
### read in two graphs

graph_files = ["/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_1/final_graph.gml"]
graph_files_2 = ["/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_2/final_graph.gml"]

graph_1, isolate_names, id_mapping = load_graphs(graph_files)
graph_2, isolate_names, id_mapping = load_graphs(graph_files_2)

sim_graph_04 = graph_1[0]
sim_graph_59 = graph_2[0]

#list(sim_graph_59.nodes(data=True))

get_seqIDs_in_nodes(sim_graph_04)
get_seqIDs_in_nodes(sim_graph_59)

total seqids in all nodes: 189804
total nodes in graph: 2988
total seqids in all nodes: 171890
total nodes in graph: 2965


{'54_7_88': 0,
 '61_11_0': 0,
 '30_2_311': 0,
 '33_8_0': 0,
 '5_3_0': 0,
 '7_17_0': 0,
 '10_11_0': 0,
 '15_10_64': 0,
 '37_14_49': 0,
 '18_2_250': 0,
 '36_5_0': 0,
 '11_2_0': 0,
 '46_10_0': 0,
 '60_6_55': 0,
 '41_1_311': 0,
 '20_2_309': 0,
 '31_1_387': 0,
 '21_1_0': 0,
 '16_16_35': 0,
 '40_9_7': 0,
 '38_13_49': 0,
 '35_0_0': 0,
 '22_9_90': 0,
 '53_6_0': 0,
 '23_2_310': 0,
 '19_1_309': 0,
 '48_8_39': 0,
 '66_12_0': 0,
 '49_5_0': 0,
 '39_13_0': 0,
 '25_2_310': 0,
 '43_9_75': 0,
 '29_1_0': 0,
 '28_8_90': 0,
 '62_1_0': 0,
 '6_5_35': 0,
 '51_0_0': 0,
 '3_9_0': 0,
 '13_17_35': 0,
 '0_0_0': 0,
 '63_16_0': 0,
 '52_16_35': 0,
 '44_0_386': 0,
 '14_1_309': 0,
 '8_5_89': 0,
 '47_17_35': 0,
 '27_9_30': 0,
 '55_1_0': 0,
 '45_2_0': 0,
 '34_2_0': 0,
 '26_4_35': 0,
 '42_6_0': 0,
 '12_3_88': 0,
 '64_17_0': 0,
 '24_3_169': 0,
 '17_13_0': 0,
 '2_4_135': 0,
 '57_15_45': 0,
 '50_17_35': 0,
 '32_0_142': 0,
 '65_0_340': 0,
 '9_11_63': 0,
 '59_7_0': 0,
 '4_2_0': 0,
 '58_19_44': 0,
 '56_16_0': 0,
 '1_3_135': 0,

In [120]:
### match clustering_ids from overall run to clustering_ids from individual runs using annotation_ids

gene_data_all = pd.read_csv('/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_all/gene_data.csv')
gene_data_g1 = pd.read_csv('/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_1/gene_data.csv')
gene_data_g2 = pd.read_csv('/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_2/gene_data.csv')

# check annotation_ids for uniqueness (to ensure that this is a suitable ID for identifying the genes with)
#combined_annotations = set(gene_data_g1['annotation_id']).union(set(gene_data_g2['annotation_id']))
#gene_annotations = set(gene_data_all['annotation_id'])
#annotations_match = combined_annotations == gene_annotations

#print(f"annotation_ids match: {annotations_match}")

#print(f"duplicated IDs: {gene_data_g1['annotation_id'].duplicated().any()}")
#print(f"duplicated IDs: {gene_data_g2['annotation_id'].duplicated().any()}")
#print(f"duplicated IDs: {gene_data_all['annotation_id'].duplicated().any()}")

# match geneIDs using annotation_id data

# rename column
gene_data_all = gene_data_all.rename(columns={'clustering_id': 'clustering_id_all'})
gene_data_g1 = gene_data_g1.rename(columns={'clustering_id': 'clustering_id_indiv'})
gene_data_g2 = gene_data_g2.rename(columns={'clustering_id': 'clustering_id_indiv'})

# first match by annotation ids:
matches_g1 = gene_data_all[['annotation_id', 'clustering_id_all']].merge(
    gene_data_g1[['annotation_id', 'clustering_id_indiv']],
    on='annotation_id',
    how='left'
)

matches_g2 = gene_data_all[['annotation_id', 'clustering_id_all']].merge(
    gene_data_g2[['annotation_id', 'clustering_id_indiv']],
    on='annotation_id',
    how='left'
)

# now drop rows where the individual seqID wasn't observed (or there's no corresponding seqID from all)
matches_g1 = matches_g1.dropna()
matches_g2 = matches_g2.dropna()

# convert to dict for faster lookup than with loc
gid_map_g1 = dict(zip(matches_g1['clustering_id_indiv'], matches_g1['clustering_id_all']))
gid_map_g2 = dict(zip(matches_g2['clustering_id_indiv'], matches_g2['clustering_id_all']))

# apply to graphs:

#print(f"before: {sim_graph_04.nodes[0].get("seqIDs", "")}")
sim_graph_04 = indSID_to_allSID(sim_graph_04, gid_map_g1)
#print(f"after: {sim_graph_04.nodes[0].get("seqIDs", "")}")
sim_graph_59 = indSID_to_allSID(sim_graph_59, gid_map_g2)

# check n of geneids
dic1 = get_seqIDs_in_nodes(sim_graph_04)
dic2 = get_seqIDs_in_nodes(sim_graph_59)

total seqids in all nodes: 189804
total nodes in graph: 2988
total seqids in all nodes: 171890
total nodes in graph: 2965


In [59]:


#### DEPRECATED


# change geneIDs to unique geneIDs (since geneIDs not unique between graphs)

def geneID_to_uGID(G, graph_id):
    
    for node in G.nodes():
        node_GIDs = G.nodes[node].get("geneIDs", "error")
        #print(f"node_GIDs: {node_GIDs}")
        node_GIDs_split = ";".join(str(f"uGID_{graph_id}_{GID}") for GID in node_GIDs.split(";"))
        G.nodes[node]["geneIDs"] = node_GIDs_split
        #print("UGIDs: ", G.nodes[node].get("geneIDs", "error"))

    return G

sim_graph_04 = geneID_to_uGID(sim_graph_04, "G1")
sim_graph_59 = geneID_to_uGID(sim_graph_59, "G2")

#print(sim_graph_04.nodes[0].get("geneIDs", "error"))
#print(sim_graph_59.nodes[3824].get("geneIDs", "error"))

In [121]:
### map nodes from ggcaller graphs to the COG labels in the centroid from pangenome

# read into df
# each "group_" refers to the centroid of that group in the pan_genomes_reference.fa
mmseqs = pd.read_csv('/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_merged/mmseqs_alignments.m8', sep='\t')

### match hits from mmseqs

# change the second graph node names to the first graph node names for nodes that match according to mmseqs

# make sure metrics are numeric
mmseqs["fident"] = pd.to_numeric(mmseqs["fident"], errors='coerce')
mmseqs["evalue"] = pd.to_numeric(mmseqs["evalue"], errors='coerce')
mmseqs["tlen"] = pd.to_numeric(mmseqs["tlen"], errors='coerce')
mmseqs["qlen"] = pd.to_numeric(mmseqs["qlen"], errors='coerce')
mmseqs["nident"] = pd.to_numeric(mmseqs["nident"], errors='coerce')

# filter for nt identity >= 98% (global) and length difference <= 5%
max_len = np.maximum(mmseqs['tlen'], mmseqs['qlen'])
nt_identity = mmseqs['nident'] / max_len  >= 0.98
nt_identity = max_len / max_len  >= 0.98
len_dif = 1-(np.abs(mmseqs['tlen'] - mmseqs['qlen']) / max_len) >= 0.95

scores = nt_identity & len_dif
mmseqs = mmseqs[scores].copy()

# iterate over target with each unique value of target, and pick the match with the highest fident; if multiple, pick the one with the smaller E value

# sort by fident (highest first) and evalue (lowest first)
mmseqs_sorted = mmseqs.sort_values(by=["fident", "evalue"], ascending=[False, True])

# only keep the first occurrence per unique target (highest fident then smallest evalue if tie)
mmseqs_filtered = mmseqs_sorted.groupby("target", as_index=False).first()

# remove "group_" prefix, keep as string
#mmseqs_filtered = mmseqs_filtered.applymap(lambda x: x.replace("group_", "") if isinstance(x, str) else x)

In [122]:
# in mmseqs, the first graph entered (in this case graph_1) is the query and the second entered (in this case graph_2) is the target
# so graph_1 is our query in mmseqs and the basegraph in the tokenized merge

# when iterating over graph_2 to append to graph_1, we want to match nodes according to their graph_1 identity
# so we need to replace all graph_2 nodes with graph_1 node ids

### THE NAME ("group_1") AND THE LABEL ('484') ARE DIFFERENT AND A NUMERIC STRING WILL CALL THE LABEL (not index)

# the groups are not the same across the two graphs!!! we match by mmseqs (that's the whole point of this)
# this chunk is just changing the node name from an integer to the group label of that node (which is originally just
# metadata within the graph)
# it doesn't map anything between the two graphs

mapping_groups_1 = dict()
for node in sim_graph_04.nodes():
    node_group = sim_graph_04.nodes[node].get("name", "error")
    #print(f"graph: 1, node_index_id: {node}, node_group_id: {node_group}")
    mapping_groups_1[int(node)] = str(node_group)
    
groupmapped_graph_1 = nx.relabel_nodes(sim_graph_04, mapping_groups_1, copy=False)

mapping_groups_2 = dict()
for node in sim_graph_59.nodes():
    node_group = sim_graph_59.nodes[node].get("name", "error")
    #print(f"graph: 1, node_index_id: {node}, node_group_id: {node_group}")
    mapping_groups_2[int(node)] = str(node_group)
    
groupmapped_graph_2 = nx.relabel_nodes(sim_graph_59, mapping_groups_2, copy=False)

### map filtered mmseqs2 hits to mapping of nodes between graphs

# mapping format: dictionary with old labels (graph_2/target groups) as keys and new labels (graph_1/query) as values

# convert df to dictionary with "target" as keys and "query" as values
# this maps groups from graph_1 to groups from graph_2
mapping = dict(zip(mmseqs_filtered["target"], mmseqs_filtered["query"]))

### to avoid matching nodes from target that have the same group_id but are not the same:
# append all nodes in query graph with _query
# append all query nodes in target graph with _query (for later matching)

# this appends _query to values (graph_1/query groups)
mapping = {key: f"{value}_query" for key, value in mapping.items()}

# relabel target graph from old labels (keys) to new labels (values, the _query-appended graph_1 groups)
# MUST SET COPY=FALSE OR NODES NOT IN MAPPING WILL BE DROPPED
relabeled_graph_2 = nx.relabel_nodes(groupmapped_graph_2, mapping, copy=False)

#print(relabeled_graph_2.nodes())

# relabel query graph
mapping_query = dict(zip(groupmapped_graph_1.nodes, groupmapped_graph_1.nodes))

mapping_query = {key: f"{value}_query" for key, value in mapping_query.items()}

relabeled_graph_1 = nx.relabel_nodes(groupmapped_graph_1, mapping_query, copy=False)

### need to do this to edges as well
#print(relabeled_graph_1.edges)
#print(relabeled_graph_2.edges)
# looks like this is done automatically

# now we can modify the tokenized code to iterate like usual, adding new node if string doesn't contain "_query"
# and merging the nodes that both end in "_query"

In [123]:
# to double-check above
if set(relabeled_graph_1.nodes) == set(relabeled_graph_2.nodes):
    print("The node sets are identical.")
else:
    print("The node sets are different.")
    
nodes_1 = set(relabeled_graph_1.nodes)
nodes_2 = set(relabeled_graph_2.nodes)

common_nodes = nodes_1 & nodes_2  # Intersection (nodes in both graphs)
only_in_graph_1 = nodes_1 - nodes_2  # Nodes only in Graph 1
only_in_graph_2 = nodes_2 - nodes_1  # Nodes only in Graph 2

#print(f"Common Nodes: {common_nodes}")
#print(f"Nodes only in Graph 1: {only_in_graph_1}")
#print(f"Nodes only in Graph 2: {only_in_graph_2}")

print(f"Common Nodes: {len(common_nodes)}")
print(f"Nodes only in Graph 1: {len(only_in_graph_1)}")
print(f"Nodes only in Graph 2: {len(only_in_graph_2)}")


# check seqIDs

nodes_1 = []
for node in relabeled_graph_1.nodes:
    nodes_1 += relabeled_graph_1.nodes[node]["seqIDs"]

nodes_2 = []
for node in relabeled_graph_2.nodes:
    nodes_2 += relabeled_graph_2.nodes[node]["seqIDs"]

nodes_1 = set(nodes_1)
nodes_2 = set(nodes_2)

if nodes_1 == nodes_2:
    print("The seqID sets are identical.")
else:
    print("The seqID sets are different.")

common_nodes = nodes_1 & nodes_2  # Intersection (nodes in both graphs)
only_in_graph_1 = nodes_1 - nodes_2  # Nodes only in Graph 1
only_in_graph_2 = nodes_2 - nodes_1  # Nodes only in Graph 2

print(f"Common seqIDs: {len(common_nodes)}")
print(f"seqIDs only in g1: {len(only_in_graph_1)}")
print(f"seqIDs only in g2: {len(only_in_graph_2)}")

#print(f"Common Nodes: {common_nodes}")
#print(f"seqIDs only in g1: {only_in_graph_1}")
#print(f"seqIDs only in g2: {next(iter(only_in_graph_2))}")

The node sets are different.
Common Nodes: 2777
Nodes only in Graph 1: 211
Nodes only in Graph 2: 149
The seqID sets are different.
Common seqIDs: 0
seqIDs only in g1: 189804
seqIDs only in g2: 170939


In [124]:
# add metadata when merging

### FROM PANAROO MERGE_NODES.PY

import itertools
from collections import Counter
#from .isvalid import del_dups
import numpy as np
#from intbitset import intbitset


def gen_node_iterables(G, nodes, feature, split=None):
    for n in nodes:
        if split is None:
            yield G.nodes[n][feature]
        else:
            yield G.nodes[n][feature].split(split)


def gen_edge_iterables(G, edges, feature):
    for e in edges:
        yield G[e[0]][e[1]][feature]


def temp_iter(list_list):
    for n in list_list:
        yield n


def iter_del_dups(iterable):
    seen = {}
    for f in itertools.chain.from_iterable(iterable):
        seen[f] = None
    return (list(seen.keys()))


def del_dups(iterable):
    seen = {}
    for f in iterable:
        seen[f] = None
    return (list(seen.keys()))


def merge_node_cluster(G,
                       nodes,
                       newNode,
                       multi_centroid=True,
                       check_merge_mems=True):

    if check_merge_mems:
        mem_count = Counter(
            itertools.chain.from_iterable(
                gen_node_iterables(G, nodes, 'members')))
        # genome IDs can be identical since they're being merged, so comment out this bit
        #if max(mem_count.values()) > 1:
            #raise ValueError("merging nodes with the same genome IDs!")

    # take node with most support as the 'consensus'
    nodes = sorted(nodes, key=lambda x: G.nodes[x]['size'])

    # First create a new node and combine the attributes
    dna = iter_del_dups(gen_node_iterables(G, nodes, 'dna'))
    maxLenId = 0
    max_l = 0
    for i, s in enumerate(dna):
        if len(s) >= max_l:
            max_l = len(s)
            maxLenId = i
            
    members = G.nodes[nodes[0]]['members'].copy()
    for n in nodes[1:]:
        members = list(dict.fromkeys(G.nodes[n]['members']))

    if multi_centroid:
        mergedDNA = any(gen_node_iterables(G, nodes, 'mergedDNA'))
    else:
        mergedDNA = True

    G.add_node(
        newNode,
        size=len(members),
        centroid=iter_del_dups(gen_node_iterables(G, nodes, 'centroid')),
        maxLenId=maxLenId,
        members=members,
        seqIDs=set(iter_del_dups(gen_node_iterables(G, nodes, 'seqIDs'))),
        #seqIDs=set(set(gen_node_iterables(G, nodes, 'seqIDs')) | set(gen_node_iterables(G, newNode, 'seqIDs'))),
        hasEnd=any(gen_node_iterables(G, nodes, 'hasEnd')),
        protein=iter_del_dups(gen_node_iterables(G, nodes, 'protein')),
        dna=dna,
        #geneIDs=";".join(iter_del_dups(gen_node_iterables(G, nodes, 'geneIDs', split=";"))),  # <-- added geneID merge for uGIDs
        annotation=";".join(
            iter_del_dups(gen_node_iterables(G, nodes, 'annotation',
                                             split=";"))),
        description=";".join(
            iter_del_dups(
                gen_node_iterables(G, nodes, 'description', split=";"))),
        lengths=list(
            itertools.chain.from_iterable(
                gen_node_iterables(G, nodes, 'lengths'))),
        longCentroidID=max(gen_node_iterables(G, nodes, 'longCentroidID')),
        paralog=any(gen_node_iterables(G, nodes, 'paralog')),
        mergedDNA=mergedDNA)
    if "prevCentroids" in G.nodes[nodes[0]]:
        G.nodes[newNode]['prevCentroids'] = ";".join(
            set(
                iter_del_dups(
                    gen_node_iterables(G, nodes, 'prevCentroids', split=";"))))

    # Now iterate through neighbours of each node and add them to the new node
    merge_nodes = set(nodes)
    for node in nodes:
        for neighbour in G.neighbors(node):
            if neighbour in merge_nodes: continue
            if G.has_edge(newNode, neighbour):
                G[newNode][neighbour]['members'] = list(dict.fromkeys(G[node][neighbour][
                    'members']))
                G[newNode][neighbour]['size'] = len(G[newNode][neighbour]['members'])
            else:
                G.add_edge(newNode,
                           neighbour,
                           size=G[node][neighbour]['size'],
                           members=G[node][neighbour]['members'])

    # remove old nodes from Graph
    G.remove_nodes_from(nodes)

    return G


def delete_node(G, node):
    # add in new edges
    for mem in G.nodes[node]['members']:
        mem_edges = list(
            set([e[1] for e in G.edges(node) if mem in G.edges[e]['members']]))
        if len(mem_edges) < 2: continue
        for n1, n2 in itertools.combinations(mem_edges, 2):
            if G.has_edge(n1, n2):
                G[n1][n2]['members'] |= intbitset([mem])
                G[n1][n2]['size'] = len(G[n1][n2]['members'])
            else:
                G.add_edge(n1, n2, size=1, members=intbitset([mem]))

    # now remove node
    G.remove_node(node)

    return G


def remove_member_from_node(G, node, member):

    # add in replacement edges if required
    mem_edges = list(
        set([e[1] for e in G.edges(node) if member in G.edges[e]['members']]))
    if len(mem_edges) > 1:
        for n1, n2 in itertools.combinations(mem_edges, 2):
            if G.has_edge(n1, n2):
                G[n1][n2]['members'] |= intbitset([member])
                G[n1][n2]['size'] = len(G[n1][n2]['members'])
            else:
                G.add_edge(n1, n2, size=1, members=intbitset([member]))

    # remove member from node
    G.nodes[node]['members'].discard(member)
    G.nodes[node]['seqIDs'] = set([
        sid for sid in G.nodes[node]['seqIDs']
        if sid.split("_")[0] != str(member)
    ])
    G.nodes[node]['size'] -= 1

    # remove member from edges of node
    edges_to_remove = []
    for e in G.edges(node):
        if member in G.edges[e]['members']:
            if len(G.edges[e]['members']) == 1:
                edges_to_remove.append(e)
            else:
                G.edges[e]['members'].discard(member)
                G.edges[e]['size'] = len(G.edges[e]['members'])
    for e in edges_to_remove:
        G.remove_edge(*e)

    return G

In [64]:
merged_graph = relabeled_graph_1
print("num nodes (before):", len(merged_graph.nodes))
print("num edges (before):", len(merged_graph.edges))

merged_graph = nx.compose_all([relabeled_graph_1, relabeled_graph_2])

print("num nodes (after):", len(merged_graph.nodes))

print("relabeled_graph_2 edges:", len(relabeled_graph_2.edges))

print("num edges (after):", len(merged_graph.edges))

# merge the two sets of unique nodes into one set of unique nodes
for node in relabeled_graph_2.nodes:

    if merged_graph.has_node(node):
        print(f"Part of query graph: {node}")
        print(merged_graph.nodes[node]['seqIDs'])

        # merge metadata from two nodes together
        merge_node_cluster(G=merged_graph, nodes=[node, node], newNode=f"{node}_merged", )

print("num nodes (after):", len(merged_graph.nodes))

print("relabeled_graph_2 edges:", len(relabeled_graph_2.edges))

print("num edges (after):", len(merged_graph.edges))

num nodes (before): 2988
num edges (before): 3072
num nodes (after): 3137
relabeled_graph_2 edges: 3085
num edges (after): 3419
Part of query graph: group_52
['126_19_22', '132_19_3', '2_15_4', '129_11_41', '72_9_49', '43_5_41', '111_0_41', '97_2_269', '109_2_269', '27_3_176', '95_0_299', '98_1_269', '139_11_41', '83_2_268', '63_7_41', '100_1_268', '125_1_41', '102_0_41', '113_2_269', '140_13_8', '105_1_269', '18_10_42', '41_3_47', '13_1_346', '7_8_41', '133_6_41', '62_8_80', '22_13_41', '82_1_41', '54_7_47', '1_14_8', '92_1_5', '52_8_49', '20_12_41', '106_2_41', '138_3_128', '134_2_41', '136_17_41', '51_6_96', '49_9_41', '15_1_27', '12_0_184', '118_2_41', '99_10_23', '119_11_22', '53_5_41', '58_5_48', '8_1_268', '24_13_41', '78_2_269', '90_0_269', '104_0_269', '93_2_41', '5_6_41', '130_9_34', '128_2_5', '74_3_41', '79_1_41', '117_1_270', '120_1_41', '80_2_269', '37_9_71', '116_1_268', '84_2_268', '21_0_345', '112_0_41', '23_9_48']
Part of query graph: group_437
['24_2_164', '80_2_239'

In [125]:
# from panaroo __main.py__ (added function def)

def format_metadata_for_gml(G):
    # add helpful attributes and write out graph in GML format
    for node in G.nodes():
        G.nodes[node]['size'] = len(G.nodes[node]['members'])
        G.nodes[node]['centroid'] = ";".join(G.nodes[node]['centroid'])
        G.nodes[node]['dna'] = ";".join(conv_list(G.nodes[node]['dna']))
        G.nodes[node]['protein'] = ";".join(conv_list(
            G.nodes[node]['protein']))
        G.nodes[node]['genomeIDs'] = ";".join(
            [str(m) for m in G.nodes[node]['members']])
        G.nodes[node]['geneIDs'] = ";".join(G.nodes[node]['seqIDs'])
        G.nodes[node]['degrees'] = G.degree[node]
        G.nodes[node]['members'] = list(G.nodes[node]['members'])
        G.nodes[node]['seqIDs'] = list(G.nodes[node]['seqIDs'])

    for edge in G.edges():
        G.edges[edge[0], edge[1]]['genomeIDs'] = ";".join(
            [str(m) for m in G.edges[edge[0], edge[1]]['members']])
        G.edges[edge[0],
                edge[1]]['members'] = list(G.edges[edge[0],
                                                   edge[1]]['members'])
        
    return G

In [126]:
# read in graph_all

graph_all = ["/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_all/final_graph.gml"]

graph_all, isolate_names, id_mapping = load_graphs(graph_all)
graph_all = graph_all[0]

#list(graph_all.nodes(data=True))

In [115]:
###### ORIGINAL

#global merged_graph

import itertools
from collections import Counter
#from .isvalid import del_dups
import numpy as np
#from intbitset import intbitset


def gen_node_iterables(G, nodes, feature, split=None):
    for n in nodes:
        if split is None:
            yield G.nodes[n][feature]
        else:
            yield G.nodes[n][feature].split(split)

            
            
# for adding to genome ids of second graph
n_genomes_basegraph = 45

# create merged graph

def main():

    merged_graph = relabeled_graph_1
    print("num nodes (before):", len(merged_graph.nodes))

    # merge the two sets of unique nodes into one set of unique nodes
    for node in relabeled_graph_2.nodes:
        if node != "_":
            #print(node)
            # ensure string (otherwise has_node thinks that node name is the node index)
            #node = str(node)
            if merged_graph.has_node(node) == True:
                
                # add metadata
                
                #print("Part of query graph. Adding metadata.")
                #print(merged_graph.nodes[node]["seqIDs"])
                #print(type(merged_graph.nodes[node]["seqIDs"]))
                #print(type(set(merged_graph.nodes[node]["seqIDs"])))
                #print(set(merged_graph.nodes[node]["seqIDs"]))
                #print(set(merged_graph.nodes[node]["seqIDs"]))

                #print(node)

                merged_set = list(set(relabeled_graph_2.nodes[node]["seqIDs"]) | set(relabeled_graph_1.nodes[node]["seqIDs"]))
                merged_graph.nodes[node]["seqIDs"] = merged_set
                
                #merged_graph.nodes[node]['total_sim'] += ";"
                #to_append = ";".join(str(int(num) + n_genomes_basegraph) for num in sim_graph_59.nodes[node]['total_sim'].split(";"))
                #merged_graph.nodes[node]['total_sim'] += to_append

                #print("merged total (after):", merged_graph.nodes[node]['total_sim'])

                #set_old = set(merged_graph.nodes[node]['members_sim']) - set(";")    
                #set_to_add = set(sim_graph_59.nodes[node]['members_sim']) - set(";")
                #set_add_reindexed = {str(int(num) + n_genomes_basegraph) for num in set_to_add}
                #set_total = set_old | set_add_reindexed
                #set_total_str = ";".join(set_total)
                #merged_graph.nodes[node]['members_sim'] = set_total_str
                
                #print("merged members (after):", merged_graph.nodes[node]['members_sim'])
                
            if merged_graph.has_node(node) == False:
                
                #print("Add new node to graph.")
                
                merged_graph.add_node(node,
                                     seqIDs=relabeled_graph_2.nodes[node]["seqIDs"])
                #members_sim=[],
                #total_sim=[])

                #### REINDEX
                
                #members_sim_unindexed = sim_graph_59.nodes[node]['members_sim']
                #print("members_sim_unindexed:", members_sim_unindexed)
                #members_sim_indexed = ";".join(str(int(num) + n_genomes_basegraph) for num in members_sim_unindexed.split(";"))
                #merged_graph.nodes[node]['members_sim'] = members_sim_indexed
                #print("members sim indexed", merged_graph.nodes[node]['members_sim'])

                #total_sim_unindexed = sim_graph_59.nodes[node]['total_sim']
                #print("total_sim_unindexed:", total_sim_unindexed)
                #total_sim_indexed = ";".join(str(int(num) + n_genomes_basegraph) for num in total_sim_unindexed.split(";"))
                #merged_graph.nodes[node]['total_sim'] = total_sim_indexed
                #print("total_sim indexed:", merged_graph.nodes[node]['total_sim'])

    print("num nodes (after):", len(merged_graph.nodes))

    print("relabeled_graph_2 edges:", len(relabeled_graph_2.edges))
    print("num edges (before):", len(merged_graph.edges))

    # merge the two sets of edges into one set of edges,
    # removing duplicates (including those with different strandedness)

    for edge in relabeled_graph_2.edges:
        
        #u = edge[0]
        #v = edge[1]
        
        # strip "_query" if it exists at the end of the string before interpreting order, then re-add
  #      if "_query" in u:
  #          u = u.removesuffix("_query")
  #          u = int(u)
  #          if "_query" in v:
  #              v = v.removesuffix("_query")
  #              v = int(v)

  #              # add in absolute order
  #              first = min(abs(u), abs(v))
  #              second = max(abs(u), abs(v))

                # make strings (same reason as with nodes, bc otherwise they're interpreted as indexes)
   #             first = f"{first}_query"
   #             second = f"{second}_query"

   #         if "_query" not in v:
   #             v = int(v)

                # add in absolute order
   #             first = min(abs(u), abs(v))
   #             second = max(abs(u), abs(v))

                # make strings (same reason as with nodes, bc otherwise they're interpreted as indexes)
   #             if first == abs(u):
   #                 first = f"{first}_query"

   #             if second == abs(u):
   #                 second = f"{second}_query"
            
   #         if "_query" not in u:
   #             u = int(u)
            
   #             if "_query" in v:
   #                 v = int(v)
   #         
   #                 # add in absolute order
   #                 first = min(abs(u), abs(v))
   #                 second = max(abs(u), abs(v))

                    # make strings (same reason as with nodes, bc otherwise they're interpreted as indexes)
    #                if first == abs(v):
    #                    first = f"{first}_query"
    #                    
    #                if second == abs(v):
    #                    second = f"{second}_query"
            
    #            if "_query" not in v: 
    #                v = int(v)
            
    #                # add in absolute order
    #                first = min(abs(u), abs(v))
    #                second = max(abs(u), abs(v))

                    # make strings (same reason as with nodes, bc otherwise they're interpreted as indexes)
    #                first = str(first)
    #                second = str(second)

            #if merged_graph.has_edge(first, second):
            if merged_graph.has_edge(edge[0], edge[1]):

                #print("Edge exists.")
                break
                
                #print("merged edge existing:", merged_graph.edges[edge[0], edge[1]]['members_sim'])

                #set_old = set(merged_graph.edges[edge[0], edge[1]]['members_sim']) - set(";")    
                #set_to_add = set(sim_graph_59.edges[edge[0], edge[1]]['members_sim']) - set(";")
                #set_add_reindexed = {str(int(num) + n_genomes_basegraph) for num in set_to_add}
                #set_total = set_old | set_add_reindexed
                #set_total_str = ";".join(set_total)
                #merged_graph.edges[edge[0], edge[1]]['members_sim'] = set_total_str
                
                #print("merged edge updated:", merged_graph.edges[edge[0], edge[1]]['members_sim'])

                #print("merged edge total existing:", merged_graph.edges[edge[0], edge[1]]['total_sim'])

                #merged_graph.edges[edge[0], edge[1]]['total_sim'] += ";"
                #to_append = ";".join(str(int(num) + n_genomes_basegraph) for num in sim_graph_59.edges[edge[0], edge[1]]['total_sim'].split(";"))
                #merged_graph.edges[edge[0], edge[1]]['total_sim'] += to_append
                
                #print("merged edge total updated:", merged_graph.edges[edge[0], edge[1]]['total_sim'])

                #print("strand sim before:", merged_graph.edges[edge[0], edge[1]]['strand_sim'])

                #merged_graph.edges[edge[0], edge[1]]['strand_sim'] += ";"
                #merged_graph.edges[edge[0], edge[1]]['strand_sim'] += sim_graph_59.edges[edge[0], edge[1]]['strand_sim']
                
                #print("strand sim after:", merged_graph.edges[edge[0], edge[1]]['strand_sim'])

            if not merged_graph.has_edge(edge[0], edge[1]):
                merged_graph.add_edge(edge[0], edge[1])
                
            #if not merged_graph.has_edge(first, second):
            #    merged_graph.add_edge(first, second)
                                #members_sim = [],
                                #strand_sim=[],
                                #total_sim=[])
                            
                #print("Added edge.")

                ### REINDEX (strand doesn't need to be indexed)

                #members_sim_unindexed = sim_graph_59.edges[edge[0], edge[1]]['members_sim']
                #print("members_sim_unindexed:", members_sim_unindexed)
                #members_sim_indexed = ";".join(str(int(num) + n_genomes_basegraph) for num in members_sim_unindexed.split(";"))
                #merged_graph.edges[edge[0], edge[1]]['members_sim'] = members_sim_indexed
                #print("members sim indexed", merged_graph.edges[edge[0], edge[1]]['members_sim'])

                #total_sim_unindexed = sim_graph_59.edges[edge[0], edge[1]]['total_sim']
                #print("total_sim_unindexed:", members_sim_unindexed)
                #total_sim_indexed = ";".join(str(int(num) + n_genomes_basegraph) for num in total_sim_unindexed.split(";"))
                #merged_graph.edges[edge[0], edge[1]]['total_sim'] = total_sim_indexed
                #print("total_sim indexed:", merged_graph.edges[edge[0], edge[1]]['total_sim'])

                #print("strand sim before:", sim_graph_59.edges[edge[0], edge[1]]['strand_sim'])
                #merged_graph.edges[edge[0], edge[1]]['strand_sim'] = sim_graph_59.edges[edge[0], edge[1]]['strand_sim']
                #print("strand sim after:", merged_graph.edges[edge[0], edge[1]]['strand_sim'])

    print("num edges (after):", len(merged_graph.edges))
    
    
    # to double-check above
    nodes_1 = []
    for node in merged_graph.nodes:
        nodes_1 += merged_graph.nodes[node]["seqIDs"]
        
    nodes_2 = []
    for node in graph_all.nodes:
        nodes_2 += graph_all.nodes[node]["seqIDs"]
        
    nodes_1 = set(nodes_1)
    nodes_2 = set(nodes_2)
        
    if nodes_1 == nodes_2:
        print("The seqID sets are identical.")
    else:
        print("The seqID sets are different.")

    common_nodes = nodes_1 & nodes_2  # Intersection (nodes in both graphs)
    only_in_graph_1 = nodes_1 - nodes_2  # Nodes only in Graph 1
    only_in_graph_2 = nodes_2 - nodes_1  # Nodes only in Graph 2

    print(f"Common seqIDs: {len(common_nodes)}")
    print(f"seqIDs only in merged: {len(only_in_graph_1)}")
    print(f"seqIDs only in all: {len(only_in_graph_2)}")
    
    #print(f"Common Nodes: {common_nodes}")
    print(f"seqIDs only in merged: {only_in_graph_1}")
    print(f"seqIDs only in all: {next(iter(only_in_graph_2))}")
    
    ### get list of seqIDs in each node

    #merged_graph = ["/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_merged/merged_graph.gml"]
    #merged_graph, isolate_names, id_mapping = load_graphs(merged_graph)
    #merged_graph = merged_graph[0]

    def get_seqIDs_in_nodes(G):  

        # get individual gene IDs as keys, and the cluster each geneID belongs to as its value
        dictionary = {}
        total_seqids = 0
        total_nodes = 0

        for node in G.nodes():
            seq_ids = set(G.nodes[node].get("seqIDs", "error"))
            #print(f"graph: 1, node_index_id: {node}, node_group_id: {node_group}")
            #seq_ids = list(seq_ids.split(";"))

            #print(f"gene ids: {gene_ids}")
            #print(f"len gene ids: {len(gene_ids)}")
            #print(f"node: {node}")

            total_seqids += len(seq_ids)
            total_nodes += 1

            for SID in seq_ids:
                dictionary[SID] = node

        print(f"total seqids in all nodes: {total_seqids}")
        print(f"total nodes in graph: {total_nodes}")

        return dictionary

    cluster_dict_merged = get_seqIDs_in_nodes(merged_graph)
    cluster_dict_all = get_seqIDs_in_nodes(graph_all)

    #cluster_dict_merged = get_geneIDs_in_nodes(merged_graph)
    #cluster_dict_all = get_geneIDs_in_nodes(graph_all)
    #print(cluster_dict_all)

    #print(cluster_dict_all['0_0_0'])
    #print("Key with the most values:", max_key)

    #print(cluster_dict_merged.keys())
    #print(len(cluster_dict_merged.values()), len(cluster_dict_all.values()))

    #print(cluster_dict_all)


    def dict_to_2d_array(d):
        row_keys = []
        row_values = []

        for key, value in d.items():
            # If the value is a list or tuple, assume multiple values
            if isinstance(value, (list, tuple)):
                for v in value:
                    row_keys.append(key)
                    row_values.append(v)
            else:
                row_keys.append(key)
                row_values.append(value)

        return pd.DataFrame([row_keys, row_values])

    rand_input_merged = dict_to_2d_array(cluster_dict_merged)
    rand_input_all = dict_to_2d_array(cluster_dict_all)

    # count cols
    #print(rand_input_merged.shape[1])
    #print(rand_input_all.shape[1])

    #print(rand_input_merged == rand_input_all)

    #print(rand_input_merged)
    #print(rand_input_all)

    #print(set(rand_input_merged.iloc[0].values) == set(rand_input_all.iloc[0].values))

    #shared = set(rand_input_merged.iloc[0].values) & set(rand_input_all.iloc[0].values)
    #print(len(shared))

    #print(rand_input_merged[0][0]) # seqs in row 0, clusters in row 1
    #print(rand_input_merged[0][1])
    #print(rand_input_all[0][0])
    #print(rand_input_all[0][1])
    
    # keep only columns that contain the common seqids:
    #rand_input_merged_filtered = rand_input_merged.loc[:, rand_input_merged.columns.isin(list(common_nodes))]
    #rand_input_all_filtered = rand_input_all.loc[:, rand_input_all.columns.isin(list(common_nodes))]

    rand_input_merged_filtered = rand_input_merged.loc[:, rand_input_merged.loc[0].isin(common_nodes)]
    rand_input_all_filtered = rand_input_all.loc[:, rand_input_all.loc[0].isin(common_nodes)]
    
    print(len(rand_input_merged_filtered.columns))
    print(len(rand_input_all_filtered.columns))

    #rand_input_merged_filtered = rand_input_merged.filter(like=isin(common_nodes), axis=1)
    #rand_input_all_filtered = rand_input_all.filter(like=isin(common_nodes), axis=1)
    

    # reorder merged dataset to match geneID order in all dataset
    #rand_input_merged_filtered = rand_input_merged_filtered.reindex(columns=rand_input_all_filtered.columns)
    
    # Get the row 0 values from both dataframes
    #row0_merged = rand_input_merged_filtered.loc[0]
    #row0_all = rand_input_all_filtered.loc[0]
    # Find the order of values in rand_input_all_filtered that also exist in rand_input_merged_filtered
    #common_order = row0_all[row0_all.isin(row0_merged.values)]
    # Now match those values back to the columns in rand_input_merged_filtered
    # by checking where the row 0 values are equal to the values in `common_order`
    #columns_in_order = [col for val in common_order for col in rand_input_merged_filtered.columns if rand_input_merged_filtered.at[0, col] == val]
    # Reorder columns
    #rand_input_merged_filtered = rand_input_merged_filtered.loc[:, columns_in_order]
    
    # Get desired value order from row 0 of rand_input_all_filtered
    desired_order = list(rand_input_all_filtered.iloc[0])

    # Create mapping from row 0 values in rand_input_merged_filtered to column names
    val_to_col = {val: col for col, val in zip(rand_input_merged_filtered.columns, rand_input_merged_filtered.iloc[0])}

    # Reorder columns based on desired value order
    columns_in_order = [val_to_col[val] for val in desired_order if val in val_to_col]

    # Apply the column reordering
    rand_input_merged_filtered = rand_input_merged_filtered[columns_in_order]

    #print(f"rand input new: {rand_input_merged_new}")

    print(rand_input_merged_filtered)
    print(rand_input_all_filtered)
    
    # put sorted clusters into Rand index
    ri = rand_score(rand_input_all_filtered.iloc[1], rand_input_merged_filtered.iloc[1])
    print(f"Rand Index: {ri}")

    ari = adjusted_rand_score(rand_input_all_filtered.iloc[1], rand_input_merged_filtered.iloc[1])
    print(f"Adjusted Rand Index: {ari}")

    # put sorted clusters into mutual information
    mutual_info = mutual_info_score(rand_input_all_filtered.iloc[1], rand_input_merged_filtered.iloc[1])
    print(f"Mutual Information: {mutual_info}")

    adj_mutual_info = adjusted_mutual_info_score(rand_input_all_filtered.iloc[1], rand_input_merged_filtered.iloc[1])
    print(f"Adjusted Mutual Information: {adj_mutual_info}")

    for node in merged_graph.nodes():
        merged_graph.nodes[node]['seqIDs'] = ";".join(merged_graph.nodes[node]['seqIDs'])
        
    #format_metadata_for_gml(merged_graph)
        
    #print(merged_graph.nodes[node]['seqIDs'])
        
    nx.write_gml(merged_graph, "/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_merged/merged_graph.gml")

    #return merged_graph
    
    
if __name__ == "__main__":
    main()

num nodes (before): 3137
num nodes (after): 3137
relabeled_graph_2 edges: 3085
num edges (before): 3315
num edges (after): 3315
The seqID sets are different.
Common seqIDs: 360711
seqIDs only in merged: 32
seqIDs only in all: 979
seqIDs only in merged: {'36_48_0', '55_15_0', '42_59_0', '43_27_35', '1_16_47', '53_66_0', '42_13_0', '39_31_32', '37_26_0', '140_36_0', '40_108_0', '0_35_3', '3_57_0', '9_29_3', '70_62_5', '37_54_1', '45_76_0', '15_29_0', '52_48_0', '39_74_0', '55_34_0', '70_62_6', '70_62_4', '34_2_0', '48_11_0', '25_30_0', '62_72_0', '73_60_0', '9_29_1', '9_29_2', '47_51_0', '3_54_1'}
seqIDs only in all: 41_1_74
total seqids in all nodes: 360743
total nodes in graph: 3137
total seqids in all nodes: 361690
total nodes in graph: 3120
360711
360711
           32854           32855           32857           32858   \
0         61_1_21           8_4_0        49_2_168        23_38_14   
1  group_49_query  group_49_query  group_49_query  group_49_query   

           32856         

In [ ]:
# plot merged graph

#merged_graph = nx.read_gml("/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_merged/merged_graph.gml")

plt.figure(figsize=(10, 8))  # Set the figure size
nx.draw(merged_graph, with_labels=False, node_color='red', edge_color='gray', node_size = 2)
plt.show()

# plot joint graph

graph_all = nx.read_gml("/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_all/final_graph.gml")

plt.figure(figsize=(10, 8))  # Set the figure size
nx.draw(graph_all, with_labels=False, node_color='red', edge_color='gray', node_size = 2)
plt.show()

In [ ]:
### DEPRECATED

# add metadata (geneIDs)

gene_ids = {}
    centroids_to_nodes = defaultdict(list)
    for cluster in clusters:
        for sid in cluster:
            gene_ids[sid] = cluster[0]
    all_centroids = set(list(gene_ids.values()))
    
for G in graphs:
    for node in G.nodes():
        G.nodes[node]["geneID"] = list(
            set([
                gene_ids[sid] for sid in G.nodes[node]['seqIDs']
                if "refound" not in sid
            ]))
        for sid in G.nodes[node]["centroid"]:
            centroids_to_nodes[sid].append(node)
            
#        G.nodes[node]["dna"] = [
#            centroid_to_seqs[sid][1] for sid in G.nodes[node]["centroid"]
#        ]#
#
#        G.nodes[node]["protein"] = [
#            centroid_to_seqs[sid][0] for sid in G.nodes[node]["centroid"]
#        ]
#        G.nodes[node]["longCentroidID"] = max([
#            (len(seq), sid) for seq, sid in zip(G.nodes[node]["dna"],
#                                                G.nodes[node]["centroid"])
 #       ])

In [ ]:

def load_graphs(graph_files, n_cpu=1):
    for graph_file in graph_files:
        if not os.path.isfile(graph_file):
            print("Missing:", graph_file)
            raise RuntimeError("Missing graph file!")

    graphs = [nx.read_gml(graph_file) for graph_file in graph_files]
    isolate_names = list(
        itertools.chain.from_iterable(
            [G.graph['isolateNames'] for G in graphs]))

    member_count = 0
    node_count = 0
    id_mapping = []
    for i, G in enumerate(graphs):
        id_mapping.append({})
        # relabel nodes to be consecutive integers from 1
        mapping = {}
        for n in G.nodes():
            mapping[n] = node_count
            node_count += 1
        G = nx.relabel_nodes(G, mapping, copy=True)

        # set up edge members and remove conflicts.
        #for e in G.edges():
        #    G[e[0]][e[1]]['members'] = [
        #        m + member_count for m in conv_list(G[e[0]][e[1]]['members'])
        #    ]

        # set up node parameters and remove conflicts.
        max_mem = -1
        for n in G.nodes():
            ncentroids = []
            for sid in G.nodes[n]['centroid'].split(";"):
                nid = update_sid(sid, member_count)
                id_mapping[i][sid] = nid
                if "refound" not in nid:
                    ncentroids.append(nid)
            G.nodes[n]['centroid'] = ncentroids
            new_ids = set()
            for sid in conv_list(G.nodes[n]['seqIDs']):
                nid = update_sid(sid, member_count)
                id_mapping[i][sid] = nid
                new_ids.add(nid)
            G.nodes[n]['seqIDs'] = new_ids
            G.nodes[n]['protein'] = del_dups(G.nodes[n]['protein'].replace(
                '*', 'J').split(";"))
            G.nodes[n]['dna'] = del_dups(G.nodes[n]['dna'].split(";"))
            G.nodes[n]['lengths'] = conv_list(G.nodes[n]['lengths'])
            G.nodes[n]['longCentroidID'][1] = update_sid(
                G.nodes[n]['longCentroidID'][1], member_count)
            G.nodes[n]['members'] = [m + member_count for m in conv_list(G.nodes[n]['members'])]
            max_mem = max(max_mem, max(G.nodes[n]['members']))

        member_count = max_mem + 1
        graphs[i] = G

    return graphs, isolate_names, id_mapping

In [ ]:
### DEPRECATED

# define functions to generate dna hashes from node metadata

def match_uGID_sequences(G, sequences_g1, sequences_g2):
    
    # allows for mapping sequence GIDs to node UGIDs
    # only needed for G1 and G2 components of merged graph, not for graph_all
    
    # append uGID sequence keys (non-unique geneIDs) with uGID prefix
    prefix = "uGID_G1_"
    sequences_g1 = {f"{prefix}{key}": value for key, value in sequences_g1.items()}
    prefix = "uGID_G2_"
    sequences_g2 = {f"{prefix}{key}": value for key, value in sequences_g2.items()}

    # check that uGIDs are unique between sequence datasets
    overlapping_keys = set(sequences_g1.keys()) & set(sequences_g2.keys())
    if overlapping_keys:
        raise ValueError(f"Duplicate keys found: {overlapping_keys}")
            
    # combine uGID-labeled sequences from G1 and G2
    return ({**sequences_g1, **sequences_g2})

In [ ]:
### DEPRECATED

### map geneIDs (joint graph) and uGIDs (merged graph) to DNA sequences

# read in gene sequences
sequences_all = pd.read_csv('/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_all/gene_data.csv', usecols=["clustering_id", "dna_sequence"])
#sequences_g1 = pd.read_csv('/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_1/gene_data.csv', usecols=["clustering_id", "dna_sequence"])
#sequences_g2 = pd.read_csv('/nfs/research/jlees/jacqueline/atb_analyses/merge_tests/staph_merge/graph_2/gene_data.csv', usecols=["clustering_id", "dna_sequence"])

#print(sequences_g1)

# make into dictionaries
sequences_all = dict(zip(sequences_all["clustering_id"], sequences_all["dna_sequence"]))
#sequences_g1 = dict(zip(sequences_g1["clustering_id"], sequences_g1["dna_sequence"]))
#sequences_g2 = dict(zip(sequences_g2["clustering_id"], sequences_g2["dna_sequence"]))

#print(sequences_g1["0_0_0"])

#sequences_merged = match_uGID_sequences(merged_graph, sequences_g1, sequences_g2)

#print(sequences_merged["uGID_G1_0_0_0"])

#######

#same = (Counter(sequences_all.values()) == Counter(sequences_merged.values()))
#print(same)

#print(len(sequences_all.values()), len(sequences_merged.values()))

# Convert values to tuples (in case they're lists or other unhashable types)
#values_all = set(tuple(v) for v in sequences_all.values())
#values_merged = set(tuple(v) for v in sequences_merged.values())

#only_in_all = values_all - values_merged
#only_in_merged = values_merged - values_all

#print("Values only in sequences_all:", only_in_all)
#print("Values only in sequences_merged:", only_in_merged)

In [ ]:
### DEPRECATED

### hash DNA sequences
        
### use aaHash here instead!!
        
import hashlib

sequences_all = {key: hashlib.md5(str(value).encode()).hexdigest() for key, value in sequences_all.items()}
#sequences_merged = {key: hashlib.md5(str(value).encode()).hexdigest() for key, value in sequences_merged.items()}

#print(sequences_all)
print(len(sequences_all.values()))

#print(list(sequences_merged.values()))

print(len(graph_all.nodes()))

In [ ]:
### deprecated

def get_seqIDs_in_nodes(G):  

    # get individual gene IDs as keys, and the cluster each geneID belongs to as its value
    dictionary = {}
    total_seqids = 0
    total_nodes = 0
    
    for node in G.nodes():
        seq_ids = G.nodes[node].get("seqIDs", "error")
        #print(f"graph: 1, node_index_id: {node}, node_group_id: {node_group}")
        seq_ids = list(seq_ids)
        
        #print(f"gene ids: {gene_ids}")
        #print(f"len gene ids: {len(gene_ids)}")
        #print(f"node: {node}")
        
        total_seqids += len(seq_ids)
        total_nodes += 1
        
        
        for SID in seq_ids:
            dictionary[SID] = node
     
    print(f"total geneids in all nodes: {total_seqids}")
    print(f"total nodes in graph: {total_nodes}")
        
    return dictionary

#cluster_dict_seqs_merged = get_seqIDs_in_nodes(merged_graph)
#cluster_dict_seqs_all = get_seqIDs_in_nodes(graph_all)
#print(cluster_dict_seqs_all)

#print(cluster_dict_all['0_0_0'])
#print("Key with the most values:", max_key)

#print(cluster_dict_merged.keys())
#print(len(cluster_dict_merged.values()), len(cluster_dict_all.values()))

# swap geneIDs/uGIDs for hashed DNA (remember we are mapping the two runs by hashed DNA, not by GID)

def match_hash_and_cluster(sequence, cluster):
    
    # Use keys from dict1 as the base order
    common_keys = [key for key in sequence if key in cluster]

    row1 = [sequence[key] for key in common_keys]
    row2 = [cluster[key] for key in common_keys]

    array_2d = pd.DataFrame([row1, row2])
    
    return array_2d
    
#rand_input_merged = match_hash_and_cluster(sequences_all, cluster_dict_merged)
#rand_input_all = match_hash_and_cluster(sequences_all, cluster_dict_all)

#######

# check that both lists have the same gene seqences in the same quantity

from collections import Counter

same_values = Counter(rand_input_merged.iloc[0].values) == Counter(rand_input_all.iloc[0].values)

print(same_values)

c1 = Counter(rand_input_merged.iloc[0].values)
c2 = Counter(rand_input_all.iloc[0].values)

# Show which values have different counts
diffs = {}

all_keys = set(c1.keys()).union(c2.keys())
for key in all_keys:
    if c1[key] != c2[key]:
        diffs[key] = {'merged': c1[key], 'all': c2[key]}

first_key = next(iter(diffs))
print("Differences:", diffs[first_key])
print(len(diffs))

#########

# Get the first rows as arrays
merged_row = rand_input_merged.iloc[0].values
all_row = rand_input_all.iloc[0].values

print(all_row)
print(merged_row)

# Make a copy of the column names to align
merged_columns = list(rand_input_merged.iloc[0].values)
used_indices = set()
ordered_columns = []

#print(f"merged cols: {merged_columns}")

for val in all_row:
    # Find the first unused column index in rand_input_merged where the value matches
    for i, v in enumerate(merged_row):
        if v == val and i not in used_indices:
            ordered_columns.append(merged_columns[i])
            used_indices.add(i)
            break  # move to the next value in all_row

#print(ordered_columns)
# Reorder the columns
rand_input_merged_new = rand_input_merged[ordered_columns]


# Get the order of columns by sorting the values in the first row (row 0)
#sorted_columns = rand_input_merged.iloc[0].sort_values().index
#print(sorted_columns)

# Reorder the DataFrame columns based on that
#rand_input_merged = rand_input_merged[sorted_columns]

# reorder merged dataset to match order of dna hashes in all dataset
#rand_input_merged_new = rand_input_merged.reindex(columns=rand_input_all.columns)

#print(f"rand input new: {rand_input_merged_new}")

# put sorted clusters into Rand index
ri = rand_score(rand_input_all.iloc[1], rand_input_merged_new.iloc[1])
print(f"Rand Index: {ri}")

ari = adjusted_rand_score(rand_input_all.iloc[1], rand_input_merged.iloc[1])
print(f"Adjusted Rand Index: {ari}")

# put sorted clusters into mutual information
mutual_info = mutual_info_score(rand_input_all.iloc[1], rand_input_merged.iloc[1])
print(f"Mutual Information: {mutual_info}")

adj_mutual_info = adjusted_mutual_info_score(rand_input_all.iloc[1], rand_input_merged.iloc[1])
print(f"Adjusted Mutual Information: {adj_mutual_info}")

In [ ]:
# context search

G.neighbors() # from nx